In [1]:
import numpy as np
import tensorflow as tf

def make_val(train_data, val_size):

    val_idx = np.random.choice(range(len(train_data[0])), val_size, replace=False)
    val_data = (train_data[0][val_idx], train_data[1][val_idx])
    train_data = (np.delete(train_data[0], val_idx, 0), np.delete(train_data[1], val_idx, 0))

    return train_data, val_data

def make_onehot(dataset, cate_num):

    dataset = (dataset[0], np.squeeze(np.eye(cate_num)[dataset[1]]))

    return dataset

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_data, test_data = tf.keras.datasets.cifar10.load_data()

train_data, val_data = make_val(train_data, 2000)

train_data = make_onehot(train_data, 10) # Shape: ((48000, 32, 32, 3), (48000, 10))
val_data = make_onehot(val_data, 10) # Shape: ((2000, 32, 32, 3), (2000, 10))
test_data = make_onehot(test_data, 10) # Shape: ((10000, 32, 32, 3), (10000, 10))

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 10
BATCH_SIZE = 4
batch_size = tf.placeholder(tf.int64)
total_batch = len(train_data[0]) // BATCH_SIZE

In [4]:
# create the Dataset
X = tf.placeholder(tf.float32, [None, 32*32*3])
Y = tf.placeholder(tf.float32, [None, 10])

dataset = tf.data.Dataset.from_tensor_slices((X, Y)).shuffle(buffer_size=10000).batch(batch_size).repeat()

# create the iter
iter = dataset.make_initializable_iterator()
features, labels = iter.get_next()

In [5]:
# create the model
D = 32*32*3
M = 256
K = 10

W1 = tf.Variable(tf.random_normal([D, M]))
b1 = tf.Variable(tf.random_normal([M]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([M, M]))
b2 = tf.Variable(tf.random_normal([M]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([M, K]))
b3 = tf.Variable(tf.random_normal([K]))

logits = tf.matmul(L2, W3) + b3

In [6]:
# define cost/loss & optimizer & accuracy
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
# train my model
for epoch in range(training_epochs):
    print('{} Epoch Start!'.format(epoch + 1))
    sess.run(iter.initializer, feed_dict={X: np.reshape(train_data[0],(-1,32*32*3)), Y: train_data[1], batch_size: BATCH_SIZE})
    total_cost = 0
    train_accu = 0

    for i in range(total_batch):
        a, c, _ = sess.run([accuracy, cost, optimizer])
        total_cost += c
        train_accu += a

    sess.run(iter.initializer, feed_dict={X: np.reshape(val_data[0],(-1,32*32*3)), Y: val_data[1], batch_size: len(val_data[0])})
    val_accu = sess.run(accuracy)

    print("Epoch: {}, Loss: {:.4f}, train Accu: {:.4f}, val Accu: {:.4f}".format(epoch + 1, total_cost / total_batch,
                                                                                 train_accu / total_batch, val_accu))
print('Learning Finished!')

1 Epoch Start!


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,3072]
	 [[Node: Placeholder_1 = Placeholder[dtype=DT_FLOAT, shape=[?,3072], _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'Placeholder_1', defined at:
  File "C:\Users\user\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\user\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\user\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\user\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\user\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever

In [10]:
# Test model and check accuracy
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: np.reshape(test_data[0],(-1,32*32*3)), Y: test_data[1]}))

Accuracy: 0.2205
